In [ ]:
!pip install langchain
!pip install tiktoken
!pip install transformers
!pip install chromadb
!pip install tqdm
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.7 MB/s eta 0:0

In [ ]:
import pandas as pd
from google.colab import drive
import os

### Mount the drive

In [ ]:
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/IR_Project/Standardized Dataset")

Mounted at /content/gdrive


### Load the data


In [ ]:
df_no_newline = pd.read_pickle("arxiv_dataset_removed_newline_5000.pkl")
df_with_newline = pd.read_pickle("arxiv_dataset_5000.pkl")

### Split the text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(df):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=400,  # max 400 tokens per chunk
        chunk_overlap=0,
        disallowed_special=(),
    )
    papers = df['paper'].tolist()
    metadata = df['metadata'].tolist()

    texts = text_splitter.create_documents(papers, metadatas=metadata)
    return texts

In [ ]:
%%time
texts_no_newline = split_text(df_no_newline)

CPU times: user 10min 23s, sys: 2.11 s, total: 10min 25s
Wall time: 10min 38s


In [ ]:
%%time
texts_with_newline = split_text(df_with_newline)

CPU times: user 5min 48s, sys: 1.37 s, total: 5min 49s
Wall time: 5min 53s


### Benchmarking Function


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm import tqdm

def batch_add_documents(doc_list, batch_size, vector_store):
        total_docs = len(doc_list)
        with tqdm(total=total_docs, desc="Adding documents", unit="docs") as progress_bar:
            for i in range(0, total_docs, batch_size):
                batch = doc_list[i : i + batch_size]
                vector_store.add_documents(batch)
                progress_bar.update(len(batch))

def embeddings_benchmark(texts, persist_directory):

    model_kwargs = {"device": "cuda"}  # change to 'cpu' if you don't have a GPU
    embeddings_model = HuggingFaceEmbeddings(
        model_name="llmrails/ember-v1",
        model_kwargs=model_kwargs,
    )
    os.chdir("/content")

    vector_store = Chroma(
        "cs_paper_store",
        embeddings_model,
        persist_directory=persist_directory,
    )

    # Specify the batch size
    batch_size = 5300

    # Add documents in batches and show progress using tqdm
    batch_add_documents(texts, batch_size, vector_store)



### Create vector store with no newline


In [ ]:
%%time
embeddings_benchmark(texts_no_newline, persist_directory="./chroma_ember_v1_no_newline")

Adding documents:  57%|█████▋    | 137800/241159 [3:38:24<2:44:38, 10.46docs/s]

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
!cp -r ./chroma_ember_v1_no_newline /content/gdrive/MyDrive/IR_Project/

### Create vector store with newline

In [ ]:
%%time
embeddings_benchmark(df_with_newline, persist_directory="./chroma_ember_v1_with_newline")